--- Day 17: Clumsy Crucible ---

The lava starts flowing rapidly once the Lava Production Facility is operational. As you leave, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.

As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!

You land near the gradually-filling pool of lava at the base of your new lavafall. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large crucibles on wheels.

The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.

To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible from the lava pool to the machine parts factory. To do this, you need to minimize heat loss while choosing a route that doesn't require the crucible to go in a straight line for too long.

Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.

For example:

2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533

Each city block is marked by a single digit that represents the amount of heat loss if the crucible enters that block. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)

Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move at most three blocks in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.

One way to minimize heat loss is this path:

2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>

This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only 102.

Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, what is the least heat loss it can incur?

In [ ]:
file_input = open(file="./Inputs/Day17.txt", mode='r')
my_input = file_input.read()

In [ ]:
from queue import PriorityQueue

class Direction:
    UP = (-1, 0)
    DOWN = (1, 0)
    LEFT = (0, -1)
    RIGHT = (0, 1)


class State:
    last_dir = None
    dir_times = 0
    pos = (0, 0)
    
    def __init__(self, last_dir, dir_times, pos):
        self.last_dir = last_dir
        self.dir_times = dir_times
        self.pos = pos
    
    def __lt__(self, other):
        return self.pos > other.pos
    
    def __repr__(self):
        last_d = None
        if self.last_dir == Direction.UP:
            last_d = "U"
        elif self.last_dir == Direction.DOWN:
             last_d = "D"
        elif self.last_dir == Direction.LEFT:
             last_d = "L"
        elif self.last_dir == Direction.RIGHT:
             last_d = "R"

        return f"{last_d}, {self.dir_times}, {self.pos}"
    
    def __eq__(self, other) -> bool:
        return self.pos == other.pos and self.dir_times == other.dir_times and self.last_dir == other.last_dir
    
    def __hash__(self) -> int:
        return hash((self.pos, self.dir_times, self.last_dir))


def Next(state : State, mapa):
    next_states = []
    new_directions = set()
    
    if state.last_dir == Direction.UP or state.last_dir == Direction.DOWN or state.last_dir == None:
        if state.pos[1] > 0:
            new_directions.add(Direction.LEFT)
        if state.pos[1] < len(mapa[0])-1:
            new_directions.add(Direction.RIGHT)
        
    if state.last_dir == Direction.LEFT or state.last_dir == Direction.RIGHT or state.last_dir == None:
        if state.pos[0] > 0:
            new_directions.add(Direction.UP)
        if state.pos[0] < len(mapa)-1:
            new_directions.add(Direction.DOWN)
    
    for new_dir in new_directions:
        next_states.append(State(new_dir, 1, (state.pos[0] + new_dir[0], state.pos[1] + new_dir[1])))
    
    
    if state.dir_times < 3:
        if state.last_dir == Direction.UP:
            if state.pos[0] > 0:
                adding_dir = Direction.UP
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.DOWN:
            if state.pos[0] < len(mapa)-1:
                adding_dir = Direction.DOWN
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.LEFT:
            if state.pos[1] > 0:
                adding_dir = Direction.LEFT
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.RIGHT:
            if state.pos[1] < len(mapa[0])-1:
                adding_dir = Direction.RIGHT
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
    
    return next_states
        

def manhatanDistance(pos1, pos2):
    return abs(pos2[0] - pos1[0]) + abs(pos2[1] - pos1[1])


def AStar(state : State, mapa):
    target = (len(mapa)-1, len(mapa[0])-1)
    frontier = PriorityQueue()
    expanding_state = state
    already_expanded = set()
    base_cost = 0
    
    while expanding_state.pos != target:
        for next_state in Next(expanding_state, mapa):
            if next_state in already_expanded:
                continue
            
            added_cost = mapa[next_state.pos[0]][next_state.pos[1]]
            new_cost = base_cost + added_cost
            frontier.put((new_cost + manhatanDistance(next_state.pos, target), (new_cost, next_state)))
        already_expanded.add(expanding_state)
        
        next_elem = frontier.get()
        expanding_state = next_elem[1][1]
        while expanding_state in already_expanded:
            next_elem = frontier.get()
            expanding_state = next_elem[1][1]
        base_cost = next_elem[1][0]
    
    return base_cost

mapa = []
for line in my_input.splitlines():
    row = []
    for num in line:
        row.append(int(num))
    mapa.append(row)

print(AStar(State(None, 0, (0, 0)), mapa))

--- Part Two ---

The crucibles of lava simply aren't large enough to provide an adequate supply of lava to the machine parts factory. Instead, the Elves are going to upgrade to ultra crucibles.

Ultra crucibles are even more difficult to steer than normal crucibles. Not only do they have trouble going in a straight line, but they also have trouble turning!

Once an ultra crucible starts moving in a direction, it needs to move a minimum of four blocks in that direction before it can turn (or even before it can stop at the end). However, it will eventually start to get wobbly: an ultra crucible can move a maximum of ten consecutive blocks without turning.

In the above example, an ultra crucible could follow this path to minimize heat loss:

2>>>>>>>>1323
32154535v5623
32552456v4254
34465858v5452
45466578v>>>>
143859879845v
445787698776v
363787797965v
465496798688v
456467998645v
122468686556v
254654888773v
432267465553v

In the above example, an ultra crucible would incur the minimum possible heat loss of 94.

Here's another example:

111111111111
999999999991
999999999991
999999999991
999999999991

Sadly, an ultra crucible would need to take an unfortunate path like this one:

1>>>>>>>1111
9999999v9991
9999999v9991
9999999v9991
9999999v>>>>

This route causes the ultra crucible to incur the minimum possible heat loss of 71.

Directing the ultra crucible from the lava pool to the machine parts factory, what is the least heat loss it can incur?

In [ ]:
def Next(state : State, mapa):
    next_states = []
    new_directions = set()
    
    if state.dir_times >= 4:
        if state.last_dir == Direction.UP or state.last_dir == Direction.DOWN or state.last_dir == None:
            if state.pos[1] > 0:
                new_directions.add(Direction.LEFT)
            if state.pos[1] < len(mapa[0])-1:
                new_directions.add(Direction.RIGHT)
            
        if state.last_dir == Direction.LEFT or state.last_dir == Direction.RIGHT or state.last_dir == None:
            if state.pos[0] > 0:
                new_directions.add(Direction.UP)
            if state.pos[0] < len(mapa)-1:
                new_directions.add(Direction.DOWN)
    
    for new_dir in new_directions:
        next_states.append(State(new_dir, 1, (state.pos[0] + new_dir[0], state.pos[1] + new_dir[1])))
    
    
    if state.dir_times < 10:
        if state.last_dir == Direction.UP:
            if state.pos[0] > 0:
                adding_dir = Direction.UP
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.DOWN:
            if state.pos[0] < len(mapa)-1:
                adding_dir = Direction.DOWN
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.LEFT:
            if state.pos[1] > 0:
                adding_dir = Direction.LEFT
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == Direction.RIGHT:
            if state.pos[1] < len(mapa[0])-1:
                adding_dir = Direction.RIGHT
                next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
        
        elif state.last_dir == None:
            adding_dir = Direction.RIGHT
            next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
            adding_dir = Direction.DOWN
            next_states.append(State(adding_dir, state.dir_times + 1, (state.pos[0] + adding_dir[0], state.pos[1] + adding_dir[1])))
            
    
    return next_states

print(AStar(State(None, 0, (0, 0)), mapa))